# Getting lat and long data

In [4]:
import pandas as pd
import os

In [36]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait, Select

In [21]:
chromedriver = "/Users/nathancheever/Downloads/chromedriver"  # You'll need to download one for your machine
os.environ["webdriver.chrome.driver"] = chromedriver
options = webdriver.ChromeOptions()

In [12]:
centroids = pd.read_csv('centroids.txt', sep='|', names=['x', 'y', 'id'])

In [28]:
centroids.loc[0, 'x']

1520334.1126511998

In [22]:
def init_driver():
    driver = webdriver.Chrome(chromedriver)
    driver.wait = WebDriverWait(driver, 2)
    return driver

In [23]:
link = "http://www.earthpoint.us/StatePlane.aspx"

Get the page

In [44]:
# start up a browser!
browser = init_driver()

In [45]:
browser.get(link)

In [46]:
# zone
zone_button = Select(browser.find_element_by_id("ContentPlaceHolder1_dlZone"))

In [47]:
zone_button.select_by_value("4302")

In [48]:
# select the US survey feet
button = browser.find_element_by_id("ContentPlaceHolder1_rbUnitOfMeasure_1")
button.click()

In [50]:
x_spot = browser.find_element_by_id("ContentPlaceHolder1_X")
y_spot = browser.find_element_by_id("ContentPlaceHolder1_Y")

In [51]:
# Test run with the first values in the df
x_spot.send_keys(str(centroids.loc[0, 'x']))
y_spot.send_keys(str(centroids.loc[0, 'y']))

In [52]:
# calc button
calc_button = browser.find_element_by_id("ContentPlaceHolder1_btnXYCalc_Button1")

In [53]:
calc_button.click()

In [54]:
stuff = browser.find_element_by_xpath('//*[@id="ContentPlaceHolder1_UpdatePanel4"]/table/tbody/tr[4]/td[2]')

In [55]:
stuff.text

'40.8157038°, -111.9338047°'

In [42]:
browser.quit()

#### Going thru all the coordinates to get lat and long

In [57]:
import time

In [ ]:
output = []

In [73]:
browser = init_driver()
browser.get(link)
browser.maximize_window()
zone_button = Select(browser.find_element_by_id("ContentPlaceHolder1_dlZone"))
zone_button.select_by_value("4302")
button = browser.find_element_by_id("ContentPlaceHolder1_rbUnitOfMeasure_1")
button.click()
x_spot = browser.find_element_by_id("ContentPlaceHolder1_X")
y_spot = browser.find_element_by_id("ContentPlaceHolder1_Y")
calc_button = browser.find_element_by_id("ContentPlaceHolder1_btnXYCalc_Button1")

for _, row in centroids.iterrows():
    x = str(row['x'])
    y = str(row['y'])
    if x in xes:  # This was used after we got about 100 in and it quit, see below
        continue
    id_ = row['id']
    x_spot.clear()
    y_spot.clear()
    x_spot.send_keys(x)
    y_spot.send_keys(y)
    calc_button.click()
    time.sleep(0.5)
    stuff = browser.find_element_by_xpath('//*[@id="ContentPlaceHolder1_UpdatePanel4"]/table/tbody/tr[4]/td[2]')
    lat, long = stuff.text.split(',')
    output.append((x, y, lat, long, id_))
   
 
browser.quit()

In [66]:
len(centroids)

182

In [74]:
len(output)

182

In [71]:
# Because of an unknown stall, we save out all the previously gathered x values so we don't have to cycle thru them agains
xes = [i[0] for i in output]

In [80]:
output_df = pd.DataFrame(output, columns=['old_x', 'old_y', 'new_x', 'new_y', 'id'])

In [83]:
output_df.head()

,old_x,old_y,new_x,new_y,id
0,1520334.11265,7466172.76139,40.8157038°,-111.9338047°,2.0
1,1520312.8879,7461851.08597,40.8038425°,-111.9338057°,3.0
2,1522965.00793,7461864.26871,40.8039136°,-111.9242268°,4.0
3,1525331.2973,7461715.69219,40.8035363°,-111.9156776°,5.0
4,1517492.8405,7460296.40785,40.7995376°,-111.9439635°,6.0


In [82]:
# Save it out to a Txt file
output_df.to_csv('lat_long_centroids.txt', sep='|', index=False)